### Importing necessary libraries

In [27]:
import os
import numpy as np
import pandas as pd
import matplotlib as plt
import pickle

### Declaring functions

In [35]:
#Function that loads a pickle file, returns the unpacked file.
def inspect_pickle_file(file_path):
    with open(file_path, 'rb') as file:
        content = pickle.load(file)
    return content

In [33]:
aapl_test = False #Set to == True for test case.
if aapl_test == True: #Example call for AAPL.p located in folder 'tickers'
    AAPL = inspect_pickle_file('tickers/AAPL.p') #Calls the function and returns it as an unpacked pickle file.
    print(AAPL)  # Display the structure of the file